RANDOM FOREST

In [2]:
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier


import pandas as pd

df = pd.read_csv("selected_features.csv")

In [3]:
df.head()

,circle,year,month,service_provider,value,technology,tech_yearly_sum,tech_avg,tech_value_diff,tech_yearly_share,technology_missing,value_lag1,value_lag2,churn_change,churn
0,Andaman and Nicobar Islands,2009,April,Aircel,852468,3G,7091626007,2.179738e+06,0.0,0.00012,0,0.0,0.0,0.0,0
1,Andaman and Nicobar Islands,2009,February,Aircel,852468,3G,7091626007,2.179738e+06,0.0,0.00012,0,852468.0,0.0,0.0,0
2,Andaman and Nicobar Islands,2009,January,Aircel,852468,3G,7091626007,2.179738e+06,0.0,0.00012,0,852468.0,852468.0,0.0,0
3,Andaman and Nicobar Islands,2009,March,Aircel,852468,3G,7091626007,2.179738e+06,0.0,0.00012,0,852468.0,852468.0,0.0,0
4,Andaman and Nicobar Islands,2009,May,Aircel,852468,3G,7091626007,2.179738e+06,0.0,0.00012,0,852468.0,852468.0,0.0,0


In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}  # to save each encoder

for col in ['circle', 'technology', 'service_provider', 'month']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # save encoder for later decoding


In [5]:
df.tail()

,circle,year,month,service_provider,value,technology,tech_yearly_sum,tech_avg,tech_value_diff,tech_yearly_share,technology_missing,value_lag1,value_lag2,churn_change,churn
65886,29,2025,3,26,150,2,4347862126,9.005452e+06,-284090.0,3.449971e-08,0,11722068.0,150.0,-11721918.0,1
65887,29,2025,4,26,11742282,2,4347862126,9.005452e+06,-12387806.0,2.700702e-03,0,150.0,11722068.0,11742132.0,0
65888,29,2025,4,26,150,2,4347862126,9.005452e+06,-279230.0,3.449971e-08,0,11742282.0,150.0,-11742132.0,1
65889,29,2025,7,26,11620668,2,4347862126,9.005452e+06,-12774587.0,2.672731e-03,0,150.0,11742282.0,11620518.0,0
65890,29,2025,7,26,150,2,4347862126,9.005452e+06,-290657.0,3.449971e-08,0,11620668.0,150.0,-11620518.0,1


In [6]:
from sklearn.model_selection import train_test_split

X_selected = df.drop(columns=['churn_change','churn'])
y_selected = df['churn']

X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y_selected, test_size=0.2, random_state=42, stratify=y_selected
)


In [7]:
model = RandomForestClassifier(
    n_estimators=100,     # number of trees
    random_state=42,
    max_depth=None,       # you can tune this later
    n_jobs=-1             # use all CPU cores
)

model.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [8]:
from sklearn.metrics import classification_report, roc_auc_score

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))


              precision    recall  f1-score   support

           0       0.97      0.97      0.97      7884
           1       0.96      0.95      0.96      5295

    accuracy                           0.96     13179
   macro avg       0.96      0.96      0.96     13179
weighted avg       0.96      0.96      0.96     13179

ROC-AUC Score: 0.9953871505095845


Once you have predictions (y_pred) on X_test, you can decode the numbers back to original strings:

In [9]:
for col in ['technology', 'circle', 'service_provider', 'month']:
    df[col] = label_encoders[col].inverse_transform(df[col])

In [10]:
df.head()

,circle,year,month,service_provider,value,technology,tech_yearly_sum,tech_avg,tech_value_diff,tech_yearly_share,technology_missing,value_lag1,value_lag2,churn_change,churn
0,Andaman and Nicobar Islands,2009,April,Aircel,852468,3G,7091626007,2.179738e+06,0.0,0.00012,0,0.0,0.0,0.0,0
1,Andaman and Nicobar Islands,2009,February,Aircel,852468,3G,7091626007,2.179738e+06,0.0,0.00012,0,852468.0,0.0,0.0,0
2,Andaman and Nicobar Islands,2009,January,Aircel,852468,3G,7091626007,2.179738e+06,0.0,0.00012,0,852468.0,852468.0,0.0,0
3,Andaman and Nicobar Islands,2009,March,Aircel,852468,3G,7091626007,2.179738e+06,0.0,0.00012,0,852468.0,852468.0,0.0,0
4,Andaman and Nicobar Islands,2009,May,Aircel,852468,3G,7091626007,2.179738e+06,0.0,0.00012,0,852468.0,852468.0,0.0,0


In [11]:
final_df = df[['year', 'month', 'circle', 'service_provider', 'technology', 'value','churn']]
final_df.head()

,year,month,circle,service_provider,technology,value,churn
0,2009,April,Andaman and Nicobar Islands,Aircel,3G,852468,0
1,2009,February,Andaman and Nicobar Islands,Aircel,3G,852468,0
2,2009,January,Andaman and Nicobar Islands,Aircel,3G,852468,0
3,2009,March,Andaman and Nicobar Islands,Aircel,3G,852468,0
4,2009,May,Andaman and Nicobar Islands,Aircel,3G,852468,0


In [12]:
final_df.to_csv("final_decoded_data.csv", index=False)

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "KNN": KNeighborsClassifier(),
    "SVM": SVC(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    acc = accuracy_score(y_test, pred)
    print(f"{name}: {acc:.4f}")


Logistic Regression: 0.9842
KNN: 0.9073
SVM: 0.5982
Decision Tree: 0.9699
Random Forest: 0.9638
XGBoost: 0.9718
